In [72]:
import hashlib
import libsbml

location = "iPAE1146.xml"
doc = libsbml.readSBMLFromFile(location)
model = doc.getModel()

reaction_notes = dict()
for reaction in model.getListOfReactions():
    # get subsystem information
    notes = reaction.getNotesString()
    splits = notes.split("<p>")
    subsystem_info  = splits[2][10:].strip()[:-4]
    reaction_notes[reaction.id] = subsystem_info

# convert but ignore errors (constant=false for kineticLaw parameters and differently scaled units)
doc.setLevelAndVersion(3,1, False)  
print(f"Converted model to level {doc.getLevel()}, version {doc.getVersion()}")

# enable and get groups plugin
if not doc.isPackageEnabled("groups"):
    doc.enablePackage("http://www.sbml.org/sbml/level3/version1/groups/version1", "groups", True)
groups_plugin = model.getPlugin("groups")

for reaction_id, subsystem in reaction_notes.items():
    # If not part of a subsystem, ignore
    if subsystem == "None" or not subsystem:
        continue
        
    # generate simple id from name string (needs a prefix, else it won't work)
    g_id = "s_" + hashlib.md5(subsystem.encode()).hexdigest()
    
    # find or create group for subsystem
    group = groups_plugin.getGroup(g_id)
    if not group:
        group = groups_plugin.createGroup()
        # part-of relationship
        group.setKind("partonomy")
        group.setName(subsystem)
        group.setId(g_id)
    else:
        assert(group.name == subsystem)
    # add reaction as group member    
    member = group.createMember()
    member.setIdRef(reaction_id)

out_location = f"{location.split('.')[0]}_with_groups.xml"
libsbml.writeSBMLToFile(doc, out_location)

Converted model to level 3, version 1


1